### Overview of Library Imports

Here is a list of the primary Python libraries used within this project and their uses:

- **pandas, numpy**: numerical calculations and data manipulation.
- **os, random**: file system access and generation of random numbers.
- **joblib**: saving/loading large objects and models.
- **TfidfVectorizer & cosine_similarity:** text feature extraction and measurement of similarity
- **MinMaxScaler**: scales numeric data.
- **ndcg_score**: measures recommendation ranking quality.
- **Surprise: SVD, Dataset, Reader, accuracy**: Collaborative filtering and evaluation tools.
- **train_test_split (Surprise)**: separates the data for
- **defaultdict**: a dictionary with a default value. 
- **matplotlib, seaborn**: libraries for data visualization.

In [1]:
import pandas as pd
import numpy as np
import random
import os
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import ndcg_score
from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

Creates a **models** folder if it doesn't already exist, avoiding errors if the folder is already there.

In [2]:
os.makedirs("models", exist_ok=True)

Loads IMDb datasets (title.basics, title.ratings, name.basics) from TSV files into pandas DataFrames for further processing.


In [3]:
print("[INFO] Loading IMDb datasets...")
title_basics = pd.read_csv("title.basics.tsv", sep="\t", low_memory=False)
title_ratings = pd.read_csv("title.ratings.tsv", sep="\t", low_memory=False)
name_basics = pd.read_csv("name.basics.tsv", sep="\t", low_memory=False)

[INFO] Loading IMDb datasets...


- Filters **title_basics** to include only rows where the title type is **movie**.
- Selects key columns from **title_basics**: **tconst**, **titleType**, **primaryTitle**, **startYear**, and **genres**.
- Selects key columns from **title_ratings**: **tconst**, **averageRating**, and **numVotes**.
- Selects and cleans **name_basics** by keeping **nconst**, **primaryName**, and **knownForTitles**, and drops rows with missing values.


In [4]:
title_basics = title_basics[title_basics["titleType"] == "movie"]
title_basics = title_basics[["tconst", "titleType", "primaryTitle", "startYear", "genres"]]
title_ratings = title_ratings[["tconst", "averageRating", "numVotes"]]
name_basics = name_basics[["nconst", "primaryName", "knownForTitles"]].dropna()

- Splits the **knownForTitles** column of **name_basics** into lists with comma as the separator.
- It makes each list wider so that each title is in a distinct row with one movie per line.
- Renames the column **knownForTitles** as **tconst** in order to enable matching against movie IDs across other datasets.
- It groups the dataset according to **tconst** and aggregates the first five related names as a comma-delimited string with the following lambda function:
- Renames the resultant **primaryName** column as **cast_crew** in accordance with the fact that it holds aggregated cast and crew information.

In [ ]:
name_basics["knownForTitles"] = name_basics["knownForTitles"].str.split(",")
name_basics = name_basics.explode("knownForTitles")
name_basics = name_basics.rename(columns={"knownForTitles": "tconst"})
crew_agg = name_basics.groupby("tconst")["primaryName"].apply(lambda x: ", ".join(x.head(5))).reset_index()
crew_agg = crew_agg.rename(columns={"primaryName": "cast_crew"})

- Merges **title_basics** and **title_ratings** DataFrames on the shared column **tconst** to join movie metadata with the ratings.
- Merges the output with **crew_agg** incorporating cast and crew details using a left join to preserve all movies.
Sorts the merged set by **numVotes** in descending order and retrieves the 10,000 most voted movies.
- Deletes any rows with missing essential fields like **genres**, **cast_crew**, or **startYear** in order to maintain data quality.
- Saves the last cleaned and filtered DataFrame into a CSV file with the name **cleaned_merged_top10k_movies.csv**.

In [ ]:
movies_combined_df = pd.merge(title_basics, title_ratings, on="tconst")
movies_combined_df = pd.merge(movies_combined_df, crew_agg, on="tconst", how="left")
filtered_movies_df = movies_combined_df.sort_values(by="numVotes", ascending=False).head(10000)
filtered_movies_df = filtered_movies_df.dropna(subset=["genres", "cast_crew", "startYear"])
filtered_movies_df.to_csv("cleaned_merged_top10k_movies.csv", index=False)

- Prints a header label along with the first 10 rows of the **filtered_movies_df** DataFrame as a preview of the cleaned up data.
- It creates a new figure with the specified dimensions of 10 by 5 inches for plotting.
- It uses **seaborn** to produce a histogram that depicts the **averageRating** column's distribution with 20 bins along with a KDE curve.
- Adds a title and axes labels to make the plot easier to read and more informative.
- Rearranges the layout for improved spacing and plots the last histogram using **plt.show().**

In [ ]:
print("\n[DATAFRAME HEAD - FILTERED MOVIES]\n")
print(filtered_movies_df.head(10))

plt.figure(figsize=(10, 5))
sns.histplot(filtered_movies_df['averageRating'], bins=20, kde=True)
plt.title("Distribution of Average Ratings")
plt.xlabel("Rating")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

- Prints a status message signaling the beginning of user rating simulation.
Extracts all of the distinct genres from the filtered movie dataset and sorts them.
- Creates an empty list to hold the simulated user ratings.
- For each user (1-1000), selects a few liked and disliked genres at random.
- Randomly samples 20 to 50 movies per user and then generates ratings based on matching genres:
Increases rating if liked genres are encountered.
-Lowers ranking if there is a dislike of genres.
- Ranks ratings between 0.5 and 10.0, to one decimal place.
-Stores the obtained results in a DataFrame and saves the file as **simulated_users_ratings.csv**.


In [ ]:
print("[INFO] Simulating realistic user ratings...")
unique_genres = sorted(set(",".join(filtered_movies_df["genres"]).split(",")))
user_simulated_ratings = []

for user_id in range(1, 1001):
    liked_genres = random.sample(unique_genres, k=random.randint(2, 4))
    disliked_genres = random.sample([g for g in unique_genres if g not in liked_genres], k=random.randint(1, 2))
    sampled_movies = filtered_movies_df.sample(n=random.randint(20, 50), random_state=user_id)

    for _, row in sampled_movies.iterrows():
        genres = row["genres"].split(",")
        rating = np.random.normal(loc=7.0, scale=1.0)
        if any(g in genres for g in liked_genres):
            rating += 1.0
        if any(g in genres for g in disliked_genres):
            rating -= 2.0
        rating = round(min(max(rating, 0.5), 10.0), 1)
        user_simulated_ratings.appendndndndndnd({"user_id": user_id, "primaryTitle": row["primaryTitle"], "rating": rating})

user_ratings_df = pd.DataFrame(user_simulated_ratings)
user_ratings_df.to_csv("simulated_users_ratings.csv", index=False)

- Prints a header and shows the first 10 rows of the **user_ratings_df** DataFrame as a preview of the simulated user ratings.
It establishes a new plot figure with dimensions of 10 by 5 inches.
- Plots a histogram of the distribution of user-generated **rating** using **seaborn** with 20 bins and a KDE curve for smoothness.
- Adds a title and axis labels that make the visualization informative and clear.
-Tightens layout spacing with **tight_layout()** and displays the plot with **plt.show()**.


In [ ]:
print("\n[DATAFRAME HEAD - USER RATINGS]\n")
print(user_ratings_df.head(10))

plt.figure(figsize=(10, 5))
sns.histplot(user_ratings_df['rating'], bins=20, kde=True)
plt.title("Distribution of Simulated User Ratings")
plt.xlabel("Rating")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

- Creates a TF-IDF vectorizer to feature-extract from the **genres** column based on the importance of every word within each movie's genres.
- Trains the vectorizer to the genres and converts the text to a numerical feature matrix named **genre_tfidf_matrix**.
- Does the same for the **cast_crew** column but utilizes a different vectorizer for dealing with names of actors and crew.
- Converts this data into **cast_tfidf_matrix**, which facilitates content-based filtering.
These matrices are employed to compute similarity between movies for the purpose of recommendations using text-based metadata.


In [ ]:
print("[INFO] Training TF-IDF vectorizers...")
genre_vectorizer = TfidfVectorizer(stop_words="english")
genre_tfidf_matrix = genre_vectorizer.fit_transform(filtered_movies_df["genres"])
cast_vectorizer = TfidfVectorizer(stop_words="english")
cast_tfidf_matrix = cast_vectorizer.fit_transform(filtered_movies_df["cast_crew"].fillna(""))

print("\nTF-IDF Sample Genre Features:\n", genre_vectorizer.get_feature_names_out()[:10])